In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.nn.functional import embedding
from torch.onnx.symbolic_opset9 import tensor
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from Citations.AttnGAN.eval.eval import generate
from extractor import addImagePath, textExtraction, imageExtraction

In [58]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']
max_length = 0
for subUrl in subUrlList:
    dirPath = '../Data/Instagram/Filter_' + subUrl + '.csv'
    data = pd.read_csv(dirPath)
    
    for i in range(len(data['caption'])):
        if len(str(data['caption'][i]).split()) > max_length:
            max_length = len(str(data['caption'][i]).split())
            word = data['caption'][i]
max_length

373

In [74]:
max_length = 0
word = ''
dirPath = '../Data/Oxford_HIC/oxford_hic_data.csv'
data = pd.read_csv(dirPath)
max_length = 0
index =0 
counter  = 5
# find the max word count of the text data['caption']
for i in range(len(data['caption'])):
    if len(str(data['caption'][i]).split()) > max_length:
        max_length = len(str(data['caption'][i]).split())
        word = data['caption'][i]
        index = i       
        #
print(max_length, i)
word

9729 3657846


"{Man} Once upon a time there was a lovely princess.\n\nBut she had an enchantment upon her of a fearful sort which could only \n\nbe broken by love's first kiss.\n\nShe was locked away in a castle guarded by a terrible fire-breathing \n\ndragon.\n\nMany brave knigts had attempted to free her from this dreadful prison, \n\nbut non prevailed.\n\nShe waited in the dragon's keep in the highest room of the tallest \n\ntower for her true love and true love's first kiss.\n\n{Laughing} \n\nLike that's ever gonna happen.\n\n{Paper Rusting, Toilet Flushes}\n\nWhat a load of - \n\n\n\nSomebody once told me the world is gonna roll me\n\nI ain't the sharpest tool in the shed\n\nShe was lookin' kind of dumb with her finger and her thumb\n\nIn the shape of an &quot;L&quot; on her forehead\n\nThe years start comin' and they don't stop comin'\n\nFed to the rules and hit the ground runnin'\n\nDidn't make sense not to live for fun\n\nYour brain gets smart but your head gets dumb\n\nSo much to do so much

In [73]:
data[data['caption'] == word]

,image_id,caption,funny_score
1696597,imgflip_4,{Man} Once upon a time there was a lovely prin...,1


In [4]:
# if args.img - dir == 'Oxford_HIC':
#     dirPath = '../Data/Oxford_HIC/oxford_hic_data.csv'
#     imgPath = '../Data/Oxford_HIC/oxford_img/'
# else:
dirPath = '../Data/Instagram/Filter_' + 'wendys' + '.csv'
imgPath = '../Data/Instagram/' + 'wendys' + '_img/'
# load data
data = pd.read_csv(dirPath)
data = addImagePath(data, imgPath)
# split data
train, test = train_test_split(data, test_size=0.2, random_state=42)

train_text = textExtraction(train['caption'])
train_image = imageExtraction(train['image_id'])
train_funny_score = torch.tensor(train['funny_score'].to_numpy())
test_text = textExtraction(test['caption'])
test_image = imageExtraction(test['image_id'])
test_funny_score = torch.tensor(test['funny_score'].to_numpy())

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 74/74 [00:02<00:00, 34.21it/s]


In [5]:
train_text.shape, train_image.shape, train_funny_score.shape

(torch.Size([293, 373, 768]), torch.Size([293, 64, 768]), torch.Size([293]))

In [6]:
train_dataset = torch.utils.data.TensorDataset(train_text, train_image, train_funny_score)
test_dataset = torch.utils.data.TensorDataset(test_text, test_image, test_funny_score)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
### 不確定是否為官方的 Gemini #############################################################################
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("describeai/gemini")
gemini = AutoModelForSeq2SeqLM.from_pretrained("describeai/gemini")
#######################################################################################################

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [95]:
gemini

NameError: name 'gemini' is not defined

In [7]:
### 官方的Gemma #########################################################################################
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
# gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto", revision="float16")
gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto",  torch_dtype=torch.bfloat16)
########################################################################################################

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gemma.to(device)
vocab_size = 256128  # 词汇表大小
embedding_dim = 768  # 嵌入维度，与你的图像嵌入维度相同
text_embedding = nn.Embedding(vocab_size, embedding_dim).to(device)

words = "👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha"
tokens = tokenizer(words, truncation=True, padding= 'max_length', max_length=100, return_tensors="pt")
output = text_embedding(tokens['input_ids'].to(device))

import torch
import torch.nn.functional as F

def find_closest_embeddings(x, embedding_matrix, top_k=1):
    # Normalize both the input tensor x and the embedding matrix
    x = F.normalize(x, dim=1)  # Normalize input tensor along feature dimension
    embedding_matrix = F.normalize(embedding_matrix, dim=1)  # Normalize embedding matrix
    
    # Compute cosine similarity between x and embedding matrix
    similarity = torch.matmul(x, embedding_matrix.T)  # Shape: [10, 50265]
    
    # Find top-k closest embeddings for each tensor in x
    top_k_values, top_k_indices = torch.topk(similarity, top_k, dim=1)
    
    return top_k_indices, top_k_values


# print(output.squeeze(0).shape)
top_k_indices, top_k_values = find_closest_embeddings(output.squeeze(0), text_embedding.weight)
# top_k_indices.shape
indices = tokenizer.decode(top_k_indices.squeeze(-1))
print(indices)

torch.Size([100, 768])
<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><bos>👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha


In [100]:
gemma

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [5]:
### LLM測試
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)

outputs = gemma.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\models\gemma\modeling_gemma.py:540: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<bos>Write a poem about Machine Learning.

Answer:

Step 1/2
Machine Learning is a powerful tool that can help us make sense of the world around us. It can analyze vast amounts of data and make predictions based on that data. It can also learn from our own actions and behaviors, and adapt to new situations. But what does it mean to be a machine? Is it just a computer program that can learn? Or is it something more? Maybe it's a combination of both. Maybe it's a combination of human intelligence and computer power. Maybe it's a combination of both. But whatever it is, it's a powerful tool that can help us make sense of the world around us. And it's a tool that can help us make sense of ourselves.

Step 2/2
So, what does it mean to be a machine? It means that we can learn from our own actions and behaviors, and adapt to new situations. It means that we can make sense of the world around us, and make predictions based on that data. It means that we can learn from our own experiences, and 

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_text = "Give me three best book."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids

outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\models\gemma\modeling_gemma.py:540: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<bos>Give me three best book.

Answer:

1. The Great Gatsby
2. The Catcher in the Rye
3. The Grapes of Wrath<eos>


In [58]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.self_att = nn.MultiheadAttention(768, 1)
        self.multi_att = nn.MultiheadAttention(768, 8)
        self.layer_norm = nn.LayerNorm(768)
        self.linear = nn.Linear(768, 768)
        self.linearOut = nn.Linear(768, 1)
        self.gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto",  torch_dtype=torch.bfloat16)
    
    def selfAttention(self, x):
        self_out,_ = self.self_att(x, x, x)
        self_out = self.layer_norm(self_out + x)
        return self_out
    
    def multiheadAttention(self, x):
        multi_out,_ = self.multi_att(x, x, x)
        multi_out = self.linear(multi_out)
        multi_out = self.layer_norm(multi_out + x)
        return multi_out
    
    def coAttention(self, x, y):
        # x: self, y: another
        co_out,_ = self.multi_att(x, y, y)
        co_out = self.linear(co_out)
        co_out = self.layer_norm(co_out + y)
        return co_out
    
    def feedForward(self, x):
        ff_out = self.linear(x)
        ff_out = self.layer_norm(ff_out + x)
        return ff_out
    
    def gemmaGenerate(self, x, max_new_tokens):
        with torch.no_grad():
            all_output = []
            print(x.shape[0])
            for i in range(x.shape[0]):
                print(x[i])
                output = self.gemma.generate(x[i][0], max_new_tokens= max_new_tokens)
                all_output.append(output)
        return torch.tensor(all_output)
               
    
    def forward(self, text, image):
        max_seq_len = max(text.shape[1], image.shape[1])
        text = nn.functional.pad(text, (0, 0, 0, max_seq_len - text.shape[1]))
        image = nn.functional.pad(image, (0, 0, 0, max_seq_len - image.shape[1]))
        text = text.transpose(0, 1)
        image = image.transpose(0, 1)
        
        text = self.selfAttention(text)
        text = self.feedForward(text)
        
        image = self.multiheadAttention(image)
        
        text = self.coAttention(text, image)
        image = self.coAttention(image, text)
        
        output = text + image
        output = self.feedForward(output)
        output = self.linearOut(output)
        output = output.squeeze(-1)
        output = output.transpose(0, 1).long()
        output = self.gemmaGenerate(output, 200)
        print(output.shape)
        # output = output.squeeze(-1)
        # output = output.transpose(0, 1).long()
        # print(output.shape)
        # output = output.reshape(32, -1)
        # print(output.shape)
        # output = self.gemma1(output)
        # output = self.gemma2(output)
        # print(output.shape)
        return output

In [59]:
model = Generator()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
gemma.to(device)
model

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\accelerate\utils\modeling.py:1405: UserWarning: Current model requires 4608 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generator(
  (self_att): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (multi_att): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (linear): Linear(in_features=768, out_features=768, bias=True)
  (linearOut): Linear(in_features=768, out_features=1, bias=True)
  (gemma): GemmaForCausalLM(
    (model): GemmaModel(
      (embed_tokens): Embedding(256000, 2048, padding_idx=0)
      (layers): ModuleList(
        (0-17): 18 x GemmaDecoderLayer(
          (self_attn): GemmaSdpaAttention(
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear(in_features=2048, out_features=256, bias=False)
            (v_proj): Linear(in_features=2048, out_features=256, bias=False)
            (o_proj): Linear(in_features=2048, out_features=2048,

In [60]:
a = torch.randn(2,3)
a[0]

tensor([1.0475, 0.3529, 0.8324])

In [61]:
epochs = 10
train_losses = []
test_losses = []
for epoch in range(epochs):
    train_loss = 0
    test_loss = 0
    with tqdm(train_loader, unit="batch") as tepoch:
        for text, image, funny_score in tepoch:
            optimizer.zero_grad()
            output = model(text.to(device).to(torch.float32), image.to(device).to(torch.float32))
            # output = gemma.generate(**temp_output, max_new_tokens=200)
            # print(output)

    #         loss = criterion(output, funny_score)
    #         loss.backward()
    #         optimizer.step()
    #         train_loss += loss.item()
    #         tepoch.set_postfix(loss=train_loss)
    # train_losses.append(train_loss)
    # with tqdm(test_loader, unit="batch") as tepoch:
    #     for text, image, funny_score in tepoch:
    #         output = model(text, image)
    #         loss = criterion(output, funny_score)
    #         test_loss += loss.item()
    #         tepoch.set_postfix(loss=test_loss)
    # test_losses.append(test_loss)

  0%|          | 0/10 [00:01<?, ?batch/s]

32
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

IndexError: tuple index out of range

In [39]:
output.shape
# temp = temp_output.transpose(0, 1)[1]
# temp.shape

torch.Size([32, 373])

In [106]:
temp

tensor([[ 0.639444649219513],
        [ 0.645720303058624],
        [ 0.632481873035431],
        [ 0.439957946538925],
        [ 0.508315026760101],
        [ 0.611480295658112],
        [ 0.575025856494904],
        [ 0.638554513454437],
        [ 0.545290052890778],
        [-0.294346719980240],
        [-0.652767658233643],
        [-0.837707340717316],
        [-1.048079013824463],
        [-0.468097448348999],
        [-0.354525029659271],
        [ 0.574068844318390],
        [-0.446700125932693],
        [-0.169422537088394],
        [-0.363713234663010],
        [-0.916093885898590],
        [-0.805513679981232],
        [-0.657463848590851],
        [ 0.152350604534149],
        [-0.206516295671463],
        [ 0.034146703779697],
        [-0.815440118312836],
        [-0.486833393573761],
        [-0.361155211925507],
        [-0.318530976772308],
        [-0.699815869331360],
        [-1.000678300857544],
        [ 0.249535590410233],
        [ 0.505719900131226],
        [ 

In [105]:
temp
decoded_token_ids = embeddings_to_tokens(temp.to(device), text_embedding.to(device))
print(decoded_token_ids)
emoji = tokenizer_emoji.decode(decoded_token_ids.tolist())
emoji

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4,
        0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


'<s><s><s><s><s><s><s><s><s>......<s>......<s>.<s>......<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s>'

In [53]:
def tensor_to_text(tensor):
    # 將 tensor 轉換為 list，然後轉換為一個字符串
    tensor_list = tensor.tolist()
    tensor_text = str(tensor_list)
    return tensor_text

input_text = tensor_to_text(temp)

gemma.to(device)
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids.to(device)
outputs = gemma.generate(**input_ids, max_new_tokens=200)
emoji_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
emoji_output


'[[0.6394446492195129], [0.6457203030586243], [0.6324818730354309], [0.43995794653892517], [0.5083150267601013], [0.6114802956581116], [0.5750258564949036], [0.6385545134544373], [0.5452900528907776], [-0.29434671998023987], [-0.6527676582336426], [-0.8377073407173157], [-1.048079013824463], [-0.468097448348999], [-0.35452502965927124], [0.5740688443183899], [-0.4467001259326935], [-0.16942253708839417], [-0.36371323466300964], [-0.9160938858985901], [-0.8055136799812317], [-0.6574638485908508], [0.15235060453414917], [-0.206516295671463], [0.03414670377969742], [-0.8154401183128357], [-0.486833393573761], [-0.3611552119255066], [-0.31853097677230835], [-0.6998158693313599], [-1.000678300857544], [0.24953559041023254], [0.5057199001312256], [0.15927836298942566], [0.6129420399665833], [0.021958105266094208], [0.21980902552604675], [0.46969395875930786], [0.27220845222473145], [0.6005236506462097], [0.5377832651138306], [0.7786491513252258], [0.49030232429504395], [0.49472081661224365],

In [93]:
str_temp = emoji_output.split("\n")
for i in range(len(str_temp)):
    str_temp[i] = str_temp[i].split(",")
#remove the last element
for i in range(len(str_temp)):
    for j in range(len(str_temp[i])):
        str_temp[i][j] = float(str_temp[i][j].replace("[", "").replace("]", "").replace(" ", ""))
    
    # if i == len(str_temp) - 1:
    #     str_temp[i] = str_temp[i][:-1]
    # else:
    #     str_temp[i] = str_temp[i]
tensor = torch.tensor(str_temp[0])

In [102]:
str_temp[0]

[0.6394446492195129,
 0.6457203030586243,
 0.6324818730354309,
 0.43995794653892517,
 0.5083150267601013,
 0.6114802956581116,
 0.5750258564949036,
 0.6385545134544373,
 0.5452900528907776,
 -0.29434671998023987,
 -0.6527676582336426,
 -0.8377073407173157,
 -1.048079013824463,
 -0.468097448348999,
 -0.35452502965927124,
 0.5740688443183899,
 -0.4467001259326935,
 -0.16942253708839417,
 -0.36371323466300964,
 -0.9160938858985901,
 -0.8055136799812317,
 -0.6574638485908508,
 0.15235060453414917,
 -0.206516295671463,
 0.03414670377969742,
 -0.8154401183128357,
 -0.486833393573761,
 -0.3611552119255066,
 -0.31853097677230835,
 -0.6998158693313599,
 -1.000678300857544,
 0.24953559041023254,
 0.5057199001312256,
 0.15927836298942566,
 0.6129420399665833,
 0.021958105266094208,
 0.21980902552604675,
 0.46969395875930786,
 0.27220845222473145,
 0.6005236506462097,
 0.5377832651138306,
 0.7786491513252258,
 0.49030232429504395,
 0.49472081661224365,
 0.3271626830101013,
 0.7254624366760254,
 0.

In [94]:
tensor = tensor.unsqueeze(-1)
print(tensor.shape)
# emoji_output
# 将嵌入转换回 token IDs
decoded_token_ids = embeddings_to_tokens(tensor.to(device), text_embedding.to(device))
print(decoded_token_ids)
emoji = tokenizer_emoji.decode(decoded_token_ids.tolist())
emoji

torch.Size([64, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4,
        0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


'<s><s><s><s><s><s><s><s><s>......<s>......<s>.<s>......<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s>'

In [ ]:
# 解析字符串为 Python 列表
import ast
parsed_data = ast.literal_eval(emoji_output)

# 转换为 PyTorch 张量
tensor = torch.tensor(parsed_data, dtype=torch.float32)

    


In [91]:
def embeddings_to_tokens(embeddings, embedding_layer):
    # 获取词汇表的嵌入
    vocab_embeddings = embedding_layer.weight
    # Normalize embeddings
    embedded_vectors = F.normalize(embeddings, p=2, dim=-1)
    vocab_embeddings = F.normalize(vocab_embeddings, p=2, dim=-1)

    # 计算相似度
    similarities = torch.matmul(embedded_vectors, vocab_embeddings.T)
    
    # 找到最接近的索引
    _, closest_indices = torch.max(similarities, dim=-1)
    return closest_indices


vocab_size = 50265  # 词汇表大小
embedding_dim = 1

# 将嵌入转换回 token IDs
decoded_token_ids = embeddings_to_tokens(emoji_output, text_embedding.to(device))
print(decoded_token_ids)
emoji = tokenizer_emoji.decode(decoded_token_ids.tolist())
emoji

AttributeError: 'str' object has no attribute 'norm'

In [30]:
import torch
import torch.nn.functional as F
def embeddings_to_tokens(embeddings, embedding_layer):
    # 获取词汇表的嵌入
    vocab_embeddings = embedding_layer.weight
    print(vocab_embeddings.shape)
    # 计算每个 token 嵌入与词汇表嵌入的余弦相似度
    similarities = F.cosine_similarity(embeddings.unsqueeze(1), vocab_embeddings.unsqueeze(0), dim=-1)
    # 获取最大相似度的索引作为 token ID
    token_ids = similarities.argmax(dim=-1)
    return token_ids

vocab_size = 50265  # 词汇表大小
embedding_dim = 1

# 将嵌入转换回 token IDs
decoded_token_ids = embeddings_to_tokens(temp, text_embedding.to(device))
print(decoded_token_ids)
emoji = tokenizer_emoji.decode(decoded_token_ids.tolist())
emoji

torch.Size([50265, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4,
        0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


'<s><s><s><s><s><s><s><s><s>......<s>......<s>.<s>......<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s>'

In [49]:
emoji = tokenizer_emoji.decode(temp.tolist())
emoji

# gemma_input = tokenizer(emoji, return_tensors="pt").to(device)
# gemma_outputs = gemma.generate(**gemma_input, max_new_tokens=200)

TypeError: argument 'ids': 'float' object cannot be interpreted as an integer

In [ ]:
# decoded_preds = tokenizer.batch_decode(preds[0], skip_special_tokens=True)